# 📝 Task 15 — SQL Reporting

Instructions:
- Write SQL queries:
  1. Categories with total revenue > 120 (using HAVING).
  2. Products with revenue above average (using subquery).
  3. Region with the highest revenue (GROUP BY + Pandas .nlargest).
- Save result to `sql_report15.csv`.
- (Optional) Combine results of several SQL queries using `pd.concat`.


In [28]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('db/sales.db')
print(pd.read_sql("SELECT * FROM sales", conn))
print(pd.read_sql("SELECT * FROM products", conn))
print(pd.read_sql("SELECT * FROM regions", conn))
cat_with_tot_rev_over120 = """
    SELECT p.category, SUM(s.revenue) AS total_revenue
    FROM products p
    INNER JOIN sales s ON s.product_id = p.product_id
    GROUP BY p.category
    HAVING total_revenue > 120
"""
products_lessthen_avg = """
    SELECT p.product_name, s.revenue
    FROM products p
    INNER JOIN sales s ON p.product_id = s.product_id
    WHERE s.revenue > (SELECT AVG(revenue) FROM sales)
"""
regions_revenue = """
   SELECT r.region, SUM(s.revenue) total_revenue
   FROM sales s
   INNER JOIN regions r ON r.date = s.date
   GROUP BY r.region
"""
df1 = pd.read_sql(cat_with_tot_rev_over120, conn)
df2 = pd.read_sql(products_lessthen_avg, conn)
df3 = pd.read_sql(regions_revenue, conn)
df3 = df3.nlargest(1, 'total_revenue')
df1['source'] = 'categories'
df2['source'] = 'products'
df3['source'] = 'regions'
report = pd.concat([df1, df2, df3], ignore_index=True).fillna("N/A")
report.to_csv('csv/sales_report.csv', index=False)
conn.close()

         date  product_id  revenue
0  2025-09-01         101     50.0
1  2025-09-01         102     30.0
2  2025-09-02         101     90.0
3  2025-09-02         103     40.0
4  2025-09-03         102     60.0
5  2025-09-03         104     75.0
6  2025-09-04         105     20.0
7  2025-09-04         101     65.0
   product_id product_name     category
0         101       Laptop  Electronics
1         102        Phone  Electronics
2         103     Backpack      Fashion
3         104       Tablet  Electronics
4         105   Headphones  Accessories
         date region
0  2025-09-01  North
1  2025-09-01   East
2  2025-09-02   West
3  2025-09-02  North
4  2025-09-03  South
5  2025-09-03   East
6  2025-09-04   West
7  2025-09-04  South
